<a href="https://colab.research.google.com/github/Strix9289/Introduction-of-Deep-Learning/blob/master/overtraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 過学習
- 学習：既知のデータに対して正確に予想すること
- 目的：未知のデータに対して正確に予想すること

学習が進みすぎると、既知のデータが持つそれ自身には意味がないような統計的な性質まで学習してしまい、精度が落ちる

+ #### 正則化
    * #### L2正則化
    * #### L1正則化
    * #### ElasticNet
+ #### 早期終了
+ #### ドロップアウト


## -正則化(regularization)

使うパラメータの数をなるべく少なくなるように学習をする
→損失関数にパラメタの大きさに対するペナルティ項(正則化項)


## -L2正則化

L2正則化では、全パラメータの2乗和を正則化項として損失関数に加えます。

L2正則化では、パラメータを完全に0にすることは少ないものの、パラメータを滑らかにすることで予測精度のより良いモデルを構築します。

Kerasでは`keras.regularizers.l2`として定義されていますが、実際には各layerにregularizerとして引数で指定して用います。

なお、`keras.regularizers.l2`は引数として、正則化項に掛かる係数を指定できます。

```py
from keras.layers import Dense
from keras import regularizers
model.add(Dense(128, kernel_regularizer=regularizers.l2(0.01))
```

<small>

<参考>
$\Theta$：パラメータ、$\lambda$：係数（＝引数）
$$
    \mathrm{E}(\boldsymbol{w}) = \sum^{N}_{n=1}\sum^{K}_{k=1} t^{(n)}_{k} \ln y^{(n)}_k + \lambda \sum_{i}w_i^2
$$

</small>

## -L1正則化

L1正則化では、全パラメータの絶対値の和を正則化項として損失関数に加えます。

L1正則化ではL2正則化よりもパラメータが0になりやすいという特徴（**スパース性**）があります。

Kerasでは`keras.regularizers.l1`として定義されていますが、実際には各layerにregularizerとして引数で指定して用います。

なお、`keras.regularizers.l1`は引数として、正則化項に掛かる係数を指定できます。

```py
from keras.layers import Dense
from keras import regularizers
model.add(Dense(128, kernel_regularizer=regularizers.l1(0.01))
```
<small>

<参考>
$\boldsymbol{w}$：パラメータ、$\lambda$：係数（＝引数）
$$
    \mathrm{E}(\boldsymbol{w}) = \sum^{N}_{n=1}\sum^{K}_{k=1} t^{(n)}_{k} \ln y^{(n)}_k + \lambda \sum_{i}|w_i|
$$

</small>

## -ElasticNet

L1正則化とL2正則化の組み合わせです。

Kerasでは`keras.regularizers.l1_l2`として定義されていますが、実際には各layerにregularizerとして引数で指定して用います。

なお、`keras.regularizers.l1_l2`は引数として、各々の正則化項に掛かる係数を指定できます。

```py
from keras.layers import Dense
from keras import regularizers
model.add(Dense(128, kernel_regularizer=regularizers.l1_l2(l1=0.01,l2=0.01))
```
<small>

<参考>
$\boldsymbol{w}$：パラメータ、$\lambda$：係数、$\alpha$：L1正則化とL2正則化の割合
$$
    \mathrm{E}(\boldsymbol{w}) = \sum^{N}_{n=1}\sum^{K}_{k=1} t^{(n)}_{k} \ln y^{(n)}_k + \lambda \sum_{i}[\alpha|w_i|+(1-\alpha)w_i^2]
$$
よくある定義式としては上の通りですが、Kerasの実装との対応は、
$$
    l1=\lambda\alpha, \quad l2=\lambda(1-\alpha) \Leftrightarrow \lambda=l1+l2, \quad \alpha = \frac{l1}{l1+l2}
$$
</small>

## 早期終了
`Keras`ではcallbackという機能を使って各エポック毎のモデルにパラメータを保存し、

検証データのコストが大きくなったら前のエポックのパラメータを使用するようにします。

`model.fit`の引数に以下のように設定します。

```py
model.fit(x=x_train, y=y_train, ...,
    callbacks=keras.callbacks.EarlyStopping(patience=0, verbose=1))
```

参考：https://keras.io/ja/callbacks/#earlystopping


## ドロップアウト
過学習で余計に学習している部分に着目すると、それは統計的なばらつきと言えるような部分でした。

こうした確率的なばらつきは、一般に大量に足し合わせると相殺されます。

そこで、訓練データセットから部分訓練データセットを大量に作成し、各モデルの予測結果を平均するアンサンブルという手法が用いられることがあります。

このアンサンブル法は大変魅力的な手法なのですが、とてつもない計算量を要するためそのまま用いることは難しいものでした。

そこで出てきたものが、**ドロップアウト (dropout)**と呼ばれる手法で、これは近似的にアンサンブル法を実現するものになっています。

具体的には、ドロップアウトは入力の一部をランダムに0にして出力するlayerの一種です。要するに一部のユニットを取り除いた状況を再現します。

このユニットの除去を確率的に行い、一部のユニットが除去された部分ネットワークに対して学習することを繰り返すことで、

多数のモデルを同時に訓練することと同じ効果を再現しているわけです。

Kerasでは、`keras.layers.core.Dropout`クラスを用いて実装できます。

```py
keras.layers.core.Dropout(rate, noise_shape=None, seed=None)
```

主な引数は、

* rate: 入力を0にする確率、0～1の実数値
* seed: 乱数のシード値

です。（入力と出力でshapeは変わりません）

参考：https://keras.io/ja/layers/core/#dropout

# バッチ学習



<ul>
    <li><strong>バッチ学習（一括学習、batch learning）</strong>
    <ul>データセット全体を一度に全て使用してパラメータを決定</ul>
    </li>
    <li><strong>オンライン学習（逐次学習、online learning）</strong>
    <ul>データセット全体を持っているが、学習を1データ毎に繰り返す</ul>
    <ul>（or データが1つずつ時系列として与えられるので、そのたびに学習を行い、パラメータを更新（ストリーム学習））</ul>
    </li>
</ul>

オンライン学習とバッチ学習を比較すると、次のような特徴があります。

<ul>
    <li>オンライン学習では1度の学習に使用するデータが1つなので、省メモリ</li>
    <li>バッチ学習ではスカラー計算ではなくベクトル・行列計算を行うので、SIMDやGPU等により高速化されやすい
    </li>
</ul>

これらの特徴は互いにトレードオフの関係となっています。

そこで、これらのいいとこどりをしようと考えられたのが、**ミニバッチ (mini-batch)**です。

データセットを少数のデータ集合、つまりミニバッチに分割し、このミニバッチごとに学習を行うということです。

ミニバッチの利用により、使用メモリ量を抑えつつ、高速なベクトル・行列計算能力を利用することができます。

そのため、MLPではミニバッチを利用した学習を行うことが標準的ですし、本講義でもミニバッチを利用した学習方法を採用しています。